# EDA Demographique

In [1]:
import polars as pl
import pyarrow.parquet as pq
import sys

filepath = "../Data/processed/sirene_infos_CLEAN.parquet" 

print("--- Début de la lecture 'bypass' ---")

try:
    print(f"Lecture du fichier via PyArrow : {filepath}")
    table_arrow = pq.read_table(
        filepath,
    )
    
    print("Conversion de la table PyArrow en DataFrame Polars...")
    df_eta_hist = pl.from_arrow(table_arrow)
    
    print("--- SUCCÈS ! ---\n")
    print("Le DataFrame est maintenant dans Polars, prêt pour la transformation.")
    print(df_eta_hist.head())

except Exception as e:
    print(f"\n--- ERREUR ---", file=sys.stderr)
    print(f"Impossible de lire le fichier, même avec PyArrow : {e}", file=sys.stderr)

--- Début de la lecture 'bypass' ---
Lecture du fichier via PyArrow : ../Data/processed/sirene_infos_CLEAN.parquet
Conversion de la table PyArrow en DataFrame Polars...
--- SUCCÈS ! ---

Le DataFrame est maintenant dans Polars, prêt pour la transformation.
shape: (5, 15)
┌───────────┬───────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────┐
│ siren     ┆ dateCreat ┆ dateFerme ┆ categorie ┆ … ┆ trancheEf ┆ caractere ┆ date_limi ┆ is_faile │
│ ---       ┆ ionUniteL ┆ ture      ┆ Juridique ┆   ┆ fectifsSi ┆ Employeur ┆ te_3_ans  ┆ d_in_3y  │
│ str       ┆ egale     ┆ ---       ┆ UniteLega ┆   ┆ ege       ┆ Siege     ┆ ---       ┆ ---      │
│           ┆ ---       ┆ date      ┆ le        ┆   ┆ ---       ┆ ---       ┆ date      ┆ i32      │
│           ┆ date      ┆           ┆ ---       ┆   ┆ str       ┆ str       ┆           ┆          │
│           ┆           ┆           ┆ i64       ┆   ┆           ┆           ┆           ┆          │
╞═══════════╪════════

---

# EDA Bilan

In [ ]:
import polars as pl
import pyarrow.parquet as pq
import sys

filepath = "../Data/processed/ICI.parquet" 

print("--- Début de la lecture 'bypass' ---")

try:
    print(f"Lecture du fichier via PyArrow : {filepath}")
    table_arrow = pq.read_table(
        filepath,
    )
    
    print("Conversion de la table PyArrow en DataFrame Polars...")
    df_eta_hist = pl.from_arrow(table_arrow)
    
    print("--- SUCCÈS ! ---\n")
    print("Le DataFrame est maintenant dans Polars, prêt pour la transformation.")
    print(df_eta_hist.head())

except Exception as e:
    print(f"\n--- ERREUR ---", file=sys.stderr)
    print(f"Impossible de lire le fichier, même avec PyArrow : {e}", file=sys.stderr)